In [1]:
import numpy as np
import os

with np.load(os.getcwd() + '/ train_age_image_data.npz', mmap_mode='c') as data:
    x_train = data['arr_0']
    y_train = data['arr_1']

num_classes = len(np.unique(y_train))

In [2]:
dim1 = x_train.shape[1]
dim2 = x_train.shape[2]
dim3 = x_train.shape[3]

In [3]:
import tensorflow as tf

def macro_f1(y_true, y_pred):
    # Convert predicted probabilities to class labels
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)

    # Calculate the number of true positive, false positive, and false negative predictions for each class
    true_positives = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=0), tf.float32)
    false_positives = tf.cast(tf.math.count_nonzero((1 - y_true) * y_pred, axis=0), tf.float32)
    false_negatives = tf.cast(tf.math.count_nonzero(y_true * (1 - y_pred), axis=0), tf.float32)

    # Calculate precision and recall for each class
    precision = true_positives / (true_positives + false_positives + 1e-6)
    recall = true_positives / (true_positives + false_negatives + 1e-6)

    # Calculate the F1 score for each class
    f1 = 2 * precision * recall / (precision + recall + 1e-6)

    # Calculate the macro-averaged F1 score by taking the mean of the F1 scores for each class
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [4]:
#from tensorflow.keras.metrics import Metric
from tensorflow.keras import layers, models
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.metrics import Accuracy

data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Rescaling(scale = 1./255, offset=0.0),
        layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(dim1, dim2, dim3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1)    
    ]
)
 
model = Sequential([
    
    data_augmentation,
    
    layers.Conv2D(32, 3, padding='valid', activation='elu', input_shape = (180,180,3)), # padding='same'
    layers.MaxPooling2D(), 
    
    layers.Conv2D(64, 3, padding='valid', activation='elu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(128, 3, padding='valid', activation='elu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(256, 3, padding='valid', activation='elu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(256, activation='elu'),
    layers.Dense(256, activation='elu'),
    layers.Dropout(0.2),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=[macro_f1])
es = EarlyStopping(monitor='macro_f1', mode = 'max', verbose = 1)

# model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# es = EarlyStopping(monitor='accuracy', mode='max', verbose=1)

In [5]:
model.fit(x_train, y_train, epochs=20, callbacks=[es])
print(model.evaluate(x_train,y_train)[1])
del x_train, y_train

Epoch 1/20
197/197 [==============================] - 162s 811ms/step - loss: 1.4565 - macro_f1: 0.3248
Epoch 2/20
197/197 [==============================] - 161s 817ms/step - loss: 1.2054 - macro_f1: 0.3384
Epoch 3/20
197/197 [==============================] - 161s 818ms/step - loss: 1.1186 - macro_f1: 0.3484
Epoch 4/20
197/197 [==============================] - 161s 819ms/step - loss: 1.0963 - macro_f1: 0.3544
Epoch 5/20
197/197 [==============================] - 162s 822ms/step - loss: 1.0566 - macro_f1: 0.3563
Epoch 6/20
197/197 [==============================] - 163s 826ms/step - loss: 1.0348 - macro_f1: 0.3580
Epoch 7/20
197/197 [==============================] - 163s 827ms/step - loss: 1.0170 - macro_f1: 0.3622
Epoch 8/20
197/197 [==============================] - 164s 830ms/step - loss: 0.9866 - macro_f1: 0.3627
Epoch 9/20
197/197 [==============================] - 163s 829ms/step - loss: 0.9760 - macro_f1: 0.3636
Epoch 10/20
197/197 [==============================] - 164s 833m

In [6]:
with np.load(os.getcwd() + '/ test_age_image_data.npz', mmap_mode='c') as data:
    x_test = data['arr_0']
    y_test = data['arr_1']

print(model.evaluate(x_test,y_test)[1])
del x_test,y_test

95/95 [==============================] - 16s 167ms/step - loss: 1.0952 - macro_f1: 0.3926
0.39255958795547485
